In [4]:
!pip install -q autogluon

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 14.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.7/222.7 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.5/454.5 kB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 382.4/382.4 kB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.3/181.3 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 12.1 MB/s eta 0:0

In [5]:
csv_path="merged_data.csv"

In [6]:
import pandas as pd
import numpy as np
import os
from sklearn.metrics import f1_score, classification_report
from autogluon.tabular import TabularPredictor


In [7]:
df = pd.read_csv(csv_path)
print(f"Shape of the dataframe: {df.shape}")
print("Head of the dataframe:")
display(df.head())

Shape of the dataframe: (5130185, 15)
Head of the dataframe:


,epoch_for_cdf_mod,proton_density,proton_bulk_speed,proton_xvelocity,proton_yvelocity,proton_zvelocity,proton_thermal,alpha_density,alpha_bulk_speed,alpha_thermal,spacecraft_xpos,spacecraft_ypos,spacecraft_zpos,CME,HALO
0,2024-05-07 00:00:27.189,-1.000000e+31,-1.000000e+31,-1.000000e+31,14.400708,1.642675,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,1.513826e+06,-624293.433449,68146.990431,0,0
1,2024-05-07 00:00:29.689,-1.000000e+31,-1.000000e+31,-1.000000e+31,15.500808,-0.794532,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,1.513826e+06,-624293.669746,68146.890206,0,0
2,2024-05-07 00:00:32.189,-1.000000e+31,-1.000000e+31,-1.000000e+31,10.233926,2.262806,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,1.513825e+06,-624293.906043,68146.789981,0,0
3,2024-05-07 00:00:34.689,-1.000000e+31,-1.000000e+31,-1.000000e+31,10.645909,5.521390,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,1.513825e+06,-624294.142357,68146.689747,0,0
4,2024-05-07 00:00:37.189,-1.000000e+31,-1.000000e+31,-1.000000e+31,18.422240,1.092300,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,1.513825e+06,-624294.378633,68146.589530,0,0


In [8]:
def load_and_preprocess(csv_path):
    """Load and preprocess the data"""
    print(f"Loading data from {csv_path}...")
    df = pd.read_csv(csv_path)
    print(f"Initial shape: {df.shape}")

    if 'epoch_for_cdf_mod' in df.columns:
        df['epoch_for_cdf_mod'] = pd.to_datetime(df['epoch_for_cdf_mod'], errors='coerce')

    df.replace(-1e31, np.nan, inplace=True)
    df = df.sort_values('epoch_for_cdf_mod')
    df.dropna(thresh=len(df.columns) * 0.7, inplace=True)
    df = df.ffill().bfill()
    print(f"After cleaning: {df.shape}")

    if 'epoch_for_cdf_mod' in df.columns:
        df['hour'] = df['epoch_for_cdf_mod'].dt.hour
        df['weekday'] = df['epoch_for_cdf_mod'].dt.weekday

    columns_to_remove = ['epoch_for_cdf_mod', 'spacecraft_xpos', 'spacecraft_ypos', 'spacecraft_zpos']
    features = [col for col in df.columns if col not in columns_to_remove + ['CME', 'HALO']]
    print(f"Feature columns: {features}")

    return df, features


In [ ]:
def run_autogluon(csv_path):
    df, features = load_and_preprocess(csv_path)

    # Split data
    split_time = df['epoch_for_cdf_mod'].quantile(0.8)
    train = df[df['epoch_for_cdf_mod'] < split_time]
    test = df[df['epoch_for_cdf_mod'] >= split_time]

    print("Training CME model...")
    predictor_cme = TabularPredictor(label='CME').fit(train[features + ['CME']])
    pred_cme = predictor_cme.predict(test[features])
    print("AutoGluon CME F1:", f1_score(test['CME'], pred_cme, average='weighted'))

    print("Training HALO model...")
    predictor_halo = TabularPredictor(label='HALO').fit(train[features + ['HALO']])
    pred_halo = predictor_halo.predict(test[features])
    print("AutoGluon HALO F1:", f1_score(test['HALO'], pred_halo, average='weighted'))
    # predictor.feature_importance(data=train)

if __name__ == "__main__":
    run_autogluon("merged_data.csv")

Loading data from merged_data.csv...
Initial shape: (5130185, 15)
After cleaning: (2041148, 15)
Feature columns: ['proton_density', 'proton_bulk_speed', 'proton_xvelocity', 'proton_yvelocity', 'proton_zvelocity', 'proton_thermal', 'alpha_density', 'alpha_bulk_speed', 'alpha_thermal', 'hour', 'weekday']


No path specified. Models will be saved in: "AutogluonModels/ag-20250709_165549"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.3.1
Python Version:     3.11.13
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sun Mar 30 16:01:29 UTC 2025
CPU Count:          2
Memory Avail:       4.55 GB / 12.67 GB (35.9%)
Disk Space Avail:   68.36 GB / 112.64 GB (60.7%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets. Defaulting to `'medium'`...
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='experimental' : New in v1.2: Pre-trained foundation model + parallel fits. The absolute best accuracy without consideration for inference speed. Does not support GPU.
	presets='best'         : Maximize accuracy. Recommended for most users. Use in competitions

Training CME model...


Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    4575.30 MB
	Train Data (Original)  Memory Usage: 124.58 MB (2.7% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Stage 5 Generators:
		Fitting DropDuplicatesFeatureGenerator...
	Types of features in original data (raw dtype, special dtypes):
		('float', []) : 9 | ['proton_density', 'proton_bulk_speed', 'proton_xvelocity', 'proton_yvelocity', 'proton_zvelocity', ...]
		('int', [])   : 2 | ['hour', 'weekday']
	Types of features in processed data (raw dtype, sp

[1000]	valid_set's binary_error: 0.144091
[2000]	valid_set's binary_error: 0.104287
[3000]	valid_set's binary_error: 0.0840784
[4000]	valid_set's binary_error: 0.070545
[5000]	valid_set's binary_error: 0.0620331
[6000]	valid_set's binary_error: 0.0571953
[7000]	valid_set's binary_error: 0.0513778
[8000]	valid_set's binary_error: 0.0477036
[9000]	valid_set's binary_error: 0.0448255
[10000]	valid_set's binary_error: 0.0422535


	0.9579	 = Validation score   (accuracy)
	1038.37s	 = Training   runtime
	11.95s	 = Validation runtime
Fitting model: LightGBM ...


[1000]	valid_set's binary_error: 0.108512
[2000]	valid_set's binary_error: 0.0654623
[3000]	valid_set's binary_error: 0.0485609
[4000]	valid_set's binary_error: 0.0415799
[5000]	valid_set's binary_error: 0.0354562
[6000]	valid_set's binary_error: 0.0311084
[7000]	valid_set's binary_error: 0.0293938
[8000]	valid_set's binary_error: 0.0272505
[9000]	valid_set's binary_error: 0.0251072
[10000]	valid_set's binary_error: 0.0239437


	0.9761	 = Validation score   (accuracy)
	949.71s	 = Training   runtime
	10.69s	 = Validation runtime
Fitting model: RandomForestGini ...
